In [1]:
# libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import preprocessing
from datetime import datetime as dt

In [2]:
#idから日付を取り出す
def get_calendar(df):
    df['calendar_date']=[i[-10:] for i in df['id']]
    return df

In [3]:
def day(df):
    df['day']=[i[-2:] for i in df['calendar_date']]
    df['day']=df['day'].astype('int')
    return df

In [4]:
def index(df):
    df=df.set_index('id')
    return df

In [5]:
#holiday_flgを土日も含めたholiday_flgに変更する
def ch_holiday(date):
    buf=date.copy()
    buf['sat_sun']=buf['day_of_week']
    holiday_dict={'Sunday': 1,
                             'Monday': 0,
                             'Tuesday': 0,
                             'Wednesday': 0,
                             'Thursday': 0,
                             'Friday': 0,
                             'Saturday': 1}
    buf=buf.replace({'sat_sun':holiday_dict})
    buf['new_holiday_flg']=buf['sat_sun']+buf['holiday_flg']
    buf.loc[buf[buf['new_holiday_flg']>0].index,"new_holiday_flg"]=1
    buf=buf.drop(['holiday_flg','sat_sun'],axis=1)
    buf=buf.rename(columns={'new_holiday_flg':'holiday_flg'})
    return buf   

In [45]:
def golden(date):
    buf=ch_holiday(date)
    buf['new_holiday_flg']=buf['holiday_flg']
    
    for i in range(1,buf.shape[0]-2):
        if (buf.loc[i-1,'new_holiday_flg']==1) & (buf.loc[i+2,'new_holiday_flg']==1) & (buf.loc[i,'new_holiday_flg']==0):
            buf.loc[i,'new_holiday_flg']=1
    
    for i in range(1,buf.shape[0]-1):
        if (buf.loc[i-1,'new_holiday_flg']==1) & (buf.loc[i+1,'new_holiday_flg']==1) & (buf.loc[i,'new_holiday_flg']==0):
            buf.loc[i,'new_holiday_flg']=1

    buf=buf.drop(['holiday_flg'],axis=1)
    buf=buf.rename(columns={'new_holiday_flg':'holiday_flg'})
    return buf   

In [46]:
#holiday_flgをmergeする
def holiday(df,date):
    buf_date=date.drop('day_of_week',axis=1)
    df=pd.merge(df,buf_date,on='calendar_date',how='left')
    return df

In [47]:
#列名を変更する
def ch_col(df):
    col=['id',
            'visitors',
            'year',
            'month',
            'day',   
            'dow',
            'holiday_flg',
            'min_visitors',
            'mean_visitors',
            'median_visitors',
            'max_visitors',
            'count_observations',
            'air_genre_name',
            'air_area_name',
            'latitude',
            'longitude',
            ]
    df1=pd.DataFrame(columns=col)
    for c in col:
        df1[c]=df[c]
    return df1

### Main

In [48]:
tr=pd.read_csv('../dataset/tr_base.csv')
te=pd.read_csv('../dataset/te_base.csv')
date=pd.read_csv('../input/date_info.csv')

In [49]:
date1=golden(date)

In [52]:
tr1=get_calendar(tr)
tr2=day(tr1)
tr3=holiday(tr2,date1)
tr4=ch_col(tr3)
tr5=index(tr4)
tr5.to_csv('../dataset/tr_tm004.csv')

In [53]:
te1=get_calendar(te)
te2=day(te1)
te3=holiday(te2,date1)
te4=ch_col(te3)
te5=index(te4)
te5.to_csv('../dataset/te_tm004.csv')